In [1]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

2024-03-04 09:39:43.630294: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-04 09:39:43.700875: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 09:39:43.701004: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 09:39:43.704502: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-04 09:39:43.720230: I tensorflow/core/platform/cpu_feature_guar

In [2]:
from PIL import Image
image = Image.open("cat.jpg")
inputs = processor(
    text=["a photo of a cat", "a photo of a dog"],
    images=image,
    return_tensors="pt",
    padding=True,
)

In [12]:
from video_processor import VideoProcessor

In [13]:
vp = VideoProcessor(
  clip_model.text_model,
  clip_model.vision_model,
  processor.tokenizer,
  processor.image_processor,
  clip_model.text_projection,
  clip_model.visual_projection,
  device = 'cpu'
) 

In [ ]:
images = vp.image_processor(
    [image,image]
)['pixel_values']

In [ ]:
images[0].shape

In [ ]:
import torch
torch.tensor(
    images
).shape

In [14]:
out = vp.extract_frames_and_audio("/DATA/sarmistha_2221cs21/basha/VideoMAE/dataset/_A5qfpLTbns.mp4")

/DATA/sarmistha_2221cs21/basha/VideoMAE/Design/video_processor.py:109: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, _ = librosa.load(video_path, sr=16000)
/DATA/sarmistha_2221cs21/anaconda3/envs/basha/lib/python3.10/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Extracting 2 sec batches


100%|██████████| 180/180 [02:35<00:00,  1.15it/s]


Extracting 2 sec batches complete


In [6]:
import pickle
with open("data.pkl",'rb') as f:
    out = pickle.load(f)

In [9]:
vp = vp.eval()

In [10]:
vp

VideoProcessor(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1

In [11]:
vision = vp("/DATA/sarmistha_2221cs21/basha/VideoMAE/dataset/_A5qfpLTbns.mp4")

/DATA/sarmistha_2221cs21/basha/VideoMAE/Design/video_processor.py:109: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, _ = librosa.load(video_path, sr=16000)
/DATA/sarmistha_2221cs21/anaconda3/envs/basha/lib/python3.10/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Extracting 2 sec batches


100%|██████████| 180/180 [02:24<00:00,  1.25it/s]
/DATA/sarmistha_2221cs21/basha/VideoMAE/Design/video_processor.py:143: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  converted_clips = torch.tensor(frames)


Extracting 2 sec batches complete
Encoding frames


100%|██████████| 180/180 [00:41<00:00,  4.35it/s]


Encoding frames done
Extracting text from audio


  0%|          | 0/180 [00:00<?, ?it/s]/DATA/sarmistha_2221cs21/anaconda3/envs/basha/lib/python3.10/site-packages/whisper/transcribe.py:113: UserWarning: Performing inference on CPU when CUDA is available
  warnings.warn("Performing inference on CPU when CUDA is available")
/DATA/sarmistha_2221cs21/anaconda3/envs/basha/lib/python3.10/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
100%|██████████| 180/180 [05:49<00:00,  1.94s/it]


Extracting text from audio completed
Encoding Text


100%|██████████| 180/180 [00:10<00:00, 17.21it/s]

Encoding text completed


In [16]:
clip_model

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

In [20]:
clip_model.text_projection(vision[0])

tensor([[-1.2062e-01,  4.5014e-02, -1.6820e-01, -3.4746e-02, -2.1113e-01,
         -1.3153e-01,  1.2401e-01, -5.0429e-01, -1.8531e-02, -2.3232e-01,
         -1.8971e-02, -3.7197e-02,  3.0465e-01,  4.0085e-01,  1.2778e-01,
          1.1729e-01, -9.8357e-02, -9.7343e-02,  3.2400e-01, -3.3887e-01,
          9.2911e-02, -2.6583e-01, -2.2583e-02, -7.6680e-02, -1.9461e-01,
         -2.8250e-01,  1.1555e-01,  4.3266e-02, -5.6809e-02, -3.5991e-02,
          4.3223e-02,  1.4839e-01,  1.7368e-01, -1.0038e-01, -1.2956e-01,
          7.6926e-02, -2.2662e-01, -1.6489e-01,  1.6700e-01,  7.5899e-02,
         -2.9313e-01, -2.8349e-01,  1.0109e-01,  8.1005e-03,  1.0431e-01,
         -8.2173e-04,  1.4887e-01, -5.7052e-02, -7.0436e-03, -1.3565e-01,
         -1.6587e-01,  3.3023e-01, -9.7329e-02,  1.3642e-01,  1.0315e-01,
         -3.0204e-01, -1.8882e-02, -1.0632e-01,  3.2190e-01, -1.3384e-03,
         -4.9162e-01, -3.6689e-01,  7.5371e-02,  4.1403e-03, -5.3402e-02,
          9.5969e-02, -1.8406e-02,  4.

In [17]:

vision[0].shape

torch.Size([1, 512])

In [ ]:
out = vp.image_model(**processor.image_processor([image,image], return_tensors="pt").to("cuda"))

In [ ]:
out.pooler_output.mean(axis=0).shape

In [ ]:
processor.tokenizer('basha')

In [ ]:

out = clip_model.text_model(
    input_ids = inputs['input_ids'],
    attention_mask = inputs['attention_mask']
)

In [ ]:
out.last_hidden_state.shape

In [ ]:
out.pooler_output

In [ ]:
out.last_hidden_state[0]